## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [102]:
rating = pd.read_csv('ratings.csv')

In [111]:
rating_group = rating.groupby('userId').count()

In [120]:
#Ищем разницу max-min
max_min = rating.groupby('userId').timestamp.max() - rating.groupby('userId').timestamp.min()
max_min.reset_index()

,userId,timestamp
0,1,97
1,2,851
2,3,71198
3,4,203560
4,5,2101
...,...,...
666,667,1014
667,668,282
668,669,685
669,670,2162705


In [126]:
#ищем пользователей у которых кол-во просмотров больше 100
rating_over_100 = rating_group.loc[(rating_group.movieId >= 100),'movieId'] 
rating_over_100.reset_index()

,userId,movieId
0,4,204
1,5,100
2,8,116
3,15,1700
4,17,363
...,...,...
258,656,128
259,659,142
260,664,519
261,665,434


In [127]:
#Джойним обе таблицы
rating_over_100.reset_index().merge(max_min.reset_index(), on = 'userId', how = 'inner')

,userId,movieId,timestamp
0,4,204,203560
1,5,100,2101
2,8,116,85187
3,15,1700,471393496
4,17,363,8053
...,...,...,...
258,656,128,3053
259,659,142,31609411
260,664,519,98180439
261,665,434,54131471


## Задание 2
Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
2. аналогичную таблицу по типам выручки с указанием адреса клиента<br>
<br>
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [128]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [129]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [130]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [131]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [132]:
#Соединяем таблицы левым джойном.
table_all = (client_base.merge(rzd, on='client_id', how = 'left')
             .merge(auto, on='client_id', how = 'left')
             .merge(air, on='client_id', how = 'left'))

In [143]:
#Вывод таблицы с типами выручки по каждому  client_id, без адреса
table_all.loc[:, table_all.columns != 'address'].fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [145]:
#аналогично, но с адресом
table_all.fillna(0)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


## Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей.<br>
Т. е. для каждого user_id известна последовательность координат (широта/долгота),<br> когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [155]:
df = pd.DataFrame(
    {'client_id' : [111, 112],
    'coordinates' : [[[55.58323763, 37.57265452], [55.61511716, 37.64543895], [55.61511716, 37.64543895]],
    [[55.58356763, 37.57865452], [55.61511716, 37.64543695], [55.45511716, 37.64567895]]]
                     })
          

In [156]:
df

,client_id,coordinates
0,111,"[[55.58323763, 37.57265452], [55.61511716, 37...."
1,112,"[[55.58356763, 37.57865452], [55.61511716, 37...."


In [162]:
df.coordinates[0][0][1]

37.57265452